In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# !unzip "/content/drive/MyDrive/Colab Notebooks/Pepper_Classification/pepper_dataset.zip" -d "/content/drive/MyDrive/Colab Notebooks/Pepper_Classification/dataset"

In [ ]:
#şimdiye kadar veri setini kaggle'dan aldık zipten çıkardık ve verilerdeki resimlere tek tek ulaşılabilmeyi sağladık
#şimdi ise amacımız driver'daki veri seti klasörünün içinde neler olduğunu doğru yüklenip yüklenmediğini anlayacağız
import os      #Python’ın işletim sistemi ile iletişim kurmamızı sağlayan kütüphanesi

base_dir = "/content/drive/MyDrive/Colab Notebooks/Pepper_Classification/dataset" #Veri setinin bulunduğu ana klasör yolunu tanımladım yani resimlerin hangi klasörde olduğunu söyledim

for mevcut_klasor, alt_klasorler, klasordeki_dosyalar in os.walk(base_dir): #verilen klasörün içini adım adım dolaşır
    print("Şu anda incelenen klasör:", mevcut_klasor) #taranan  klasörün tam yolu
    print("Bu klasörün alt klasörleri:", alt_klasorler) #o klasörün içindeki alt klasörlerin listesini gösterir
    print("Bu klasördeki dosyalar:", klasordeki_dosyalar) #Bu da klasör içinde doğrudan bulunan dosyaları gösterir
    #os.walk() fonksiyonu her zaman bir döngüyle çalıştığı için döngü kullandık ve sadece ilk katmanı görmek için break kullanıyoruz
#Bu adımı uygulamamızın amacı klasörün doğru yerde açıldığını kontrol etmek bu doğrulama adımıdır zorunlu değil ancak hataları önler

Şu anda incelenen klasör: /content/drive/MyDrive/Colab Notebooks/Pepper_Classification/dataset
Bu klasörün alt klasörleri: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy']
Bu klasördeki dosyalar: []
Şu anda incelenen klasör: /content/drive/MyDrive/Colab Notebooks/Pepper_Classification/dataset/Pepper__bell___Bacterial_spot
Bu klasörün alt klasörleri: []
Bu klasördeki dosyalar: ['09035eae-2a1c-4718-8877-5270a1fe8462___JR_B.Spot 3280.JPG', '0d8445ac-6333-42b0-bf32-950a2dd83908___NREC_B.Spot 1862.JPG', '00f2e69a-1e56-412d-8a79-fdce794a17e4___JR_B.Spot 3132.JPG', '0726c166-3471-41a8-8fe6-18600e89e802___NREC_B.Spot 1812.JPG', '1024fbd6-ded7-4532-becb-aeb624ea832b___JR_B.Spot 8884.JPG', '1181b18f-8d1d-4ba1-9fba-dfe397c585ac___NREC_B.Spot 1846.JPG', '11e6ce0a-8511-485a-b22c-21b978d28e5e___JR_B.Spot 3383.JPG', '08485974-1e38-4e34-b296-fadd9cc26942___NREC_B.Spot 9193.JPG', '0c99cb45-b4e0-4ade-bba5-fab3b678f0bb___JR_B.Spot 8912.JPG', '02baf62e-11e2-4dde-97fb-e369b57d55d3___JR_B.Spot 89

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

data_dir = "/content/drive/MyDrive/Colab Notebooks/Pepper_Classification/dataset"
output_dir = "/content/drive/MyDrive/Colab Notebooks/Pepper_Classification/split_dataset"

train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

classes = os.listdir(data_dir)

for c in classes:
    class_path = os.path.join(data_dir, c)
    imgs = [os.path.join(class_path, f) for f in os.listdir(class_path)]

    # %80 train, %20 test
    train_files, test_files = train_test_split(imgs, test_size=0.2, random_state=42)

    os.makedirs(os.path.join(train_dir, c), exist_ok=True)
    os.makedirs(os.path.join(test_dir, c), exist_ok=True)

    # Kopyalama
    for f in train_files:
        shutil.copy(f, os.path.join(train_dir, c))

    for f in test_files:
        shutil.copy(f, os.path.join(test_dir, c))
#test ve eğitim olarak 2 ye ayrıldı


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir = "/content/drive/MyDrive/Colab Notebooks/Pepper_Classification/split_dataset/train"
test_data_dir  = "/content/drive/MyDrive/Colab Notebooks/Pepper_Classification/split_dataset/test"

img_size = (256, 256)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2     # train'in içinden validation çıkarıyoruz (%80 train → %20 val)
)

train_gen = datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Test generator
test_datagen = ImageDataGenerator(rescale=1./255)

test_gen = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False  #Test sırasında etiket sırası ile tahmin sırası birebir eşleşmelidir.
)


Found 1584 images belonging to 2 classes.
Found 395 images belonging to 2 classes.
Found 496 images belonging to 2 classes.
